In [46]:
# %run ./fyers_auth.ipynb

In [47]:
from tensorflow import keras
import sys
sys.path.append('./shared')
sys.path.append('./strategies')
sys.path.append('./sql')
from data_fetching_func import dynamicDataFetching,clean_data 
from ema_crossover import ema_crossover
from money_management_rules import already_in_position,check_three_loses
import pandas as pd
import pandas_ta as ta
import datetime as dt
import numpy as np
from sql_config import connect_db,execute_and_fetch

In [48]:
db = connect_db()
cursor = db.cursor()
cursor.execute("USE tradingbot")

Importing Model

# Trend Prediction

In [86]:
startDate = (dt.date.today() - dt.timedelta(days = 800)) #last working day
endDate = dt.date.today()
resolution = "1D"
trend_df = dynamicDataFetching(fyers,startDate,endDate,symbol,resolution)
clean_data(trend_df)

In [87]:
trend_df.shape

(545, 5)

In [52]:
trend_df.tail()

,open,high,low,close,volume
date,,,,,
2022-10-17 05:30:00+05:30,99.75,100.40,98.10,99.55,37568783.0
2022-10-18 05:30:00+05:30,100.30,101.00,99.80,100.65,30722156.0
2022-10-19 05:30:00+05:30,100.60,101.25,99.70,100.05,38764542.0
2022-10-20 05:30:00+05:30,99.50,101.20,98.80,100.90,38091315.0
2022-10-21 05:30:00+05:30,100.85,101.45,99.35,100.20,30973997.0


In [53]:
# Adding indicators
trend_df['RSI']=ta.rsi(trend_df.close, length=15)
trend_df['EMA_20']=ta.ema(trend_df.close, length=20)
trend_df['EMA_100']=ta.ema(trend_df.close, length=100)
trend_df['EMA_150']=ta.ema(trend_df.close, length=150)
trend_df['EMA_200']=ta.ema(trend_df.close, length=200)
trend_df['EMA_500']=ta.ema(trend_df.close, length=500)


# Dropping null values after adding indicators and features
trend_df.dropna(inplace=True)
trend_df.reset_index(inplace = True)
# trend_df.drop(['volume', 'date'], axis=1, inplace=True) # not required
trend_df.drop([ 'date'], axis=1, inplace=True) # not required

In [54]:
trend_df = trend_df.tail(30)
trend_df.tail()


,open,high,low,close,volume,RSI,EMA_20,EMA_100,EMA_150,EMA_200,EMA_500
183,99.75,100.40,98.10,99.55,37568783.0,43.463089,101.512288,104.563477,106.685317,107.739421,93.063521
184,100.30,101.00,99.80,100.65,30722156.0,46.485680,101.430165,104.485982,106.605379,107.668879,93.093806
185,100.60,101.25,99.70,100.05,38764542.0,45.077278,101.298721,104.398141,106.518553,107.593069,93.121575
186,99.50,101.20,98.80,100.90,38091315.0,47.491982,101.260748,104.328871,106.444135,107.526472,93.152627
187,100.85,101.45,99.35,100.20,30973997.0,45.718424,101.159724,104.247111,106.361431,107.453571,93.180760


In [55]:
trend_df.shape

(30, 11)

In [56]:
# Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled = sc.fit_transform(trend_df)
print(data_set_scaled)

[[0.86363636 0.75697211 0.83665339 0.75268817 0.41654363 0.71518736
  0.91741449 0.93926001 1.         1.         0.        ]
 [0.84710744 0.80478088 0.91633466 0.89964158 0.12798515 0.88565673
  0.93879432 0.95340541 0.99533807 0.99162203 0.05773305]
 [0.94628099 0.86454183 1.         0.89964158 0.17025133 0.88565673
  0.95813797 0.96727069 0.99073789 0.98332741 0.11523563]
 [0.80578512 1.         0.86055777 1.         0.70414154 1.
  0.99874215 0.99394846 0.9948524  0.98254102 0.17767279]
 [1.         0.88844622 0.91633466 0.84587814 0.40742996 0.78354788
  1.         1.         0.98562271 0.97035872 0.23193007]
 [0.79752066 0.7689243  0.79681275 0.75268817 0.97724979 0.6639081
  0.97968547 0.99377946 0.96847962 0.95140236 0.28117549]
 [0.79752066 0.66533865 0.6374502  0.56630824 0.25839502 0.44892375
  0.9184005  0.96337759 0.93549233 0.91884408 0.32063379]
 [0.6322314  0.67729084 0.71713147 0.75985663 0.27096643 0.69314832
  0.90750755 0.95881704 0.91963135 0.90093071 0.36989397]
 

In [57]:
data_set_scaled = np.reshape(data_set_scaled, (1, 30, 11))

In [58]:
data_set_scaled.shape

(1, 30, 11)

In [49]:
model = keras.models.load_model('./models/lstm_trend_1.0/')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 first_layer (LSTM)          (None, 150)               97200     
                                                                 
 dense_layer (Dense)         (None, 1)                 151       
                                                                 
Total params: 97,351
Trainable params: 97,351
Non-trainable params: 0
_________________________________________________________________


In [59]:
y_pred = model.predict(data_set_scaled)
y_pred.shape

1/1 [==============================] - 0s 232ms/step


(1, 1)

In [99]:
trend_signal = int(y_pred)
if trend_signal == 0 :
    trend_signal = -1 #DOWN TREND
elif trend_signal == 1:
    trend_signal = 1 #UP TREND

cursor.execute(f"INSERT INTO trend VALUES(curdate(),'{symbol}',{trend_signal})")
db.commit()
# TODO - store in some kind of db then retrieve afterwards when required

# Techinal Strategy

Data fetching for techinal indicator


In [88]:

startDate = (dt.date.today() - dt.timedelta(days = 1)) #if week day

if  dt.date.today().weekday() == 6: #if sunday
    startDate = (dt.date.today() - dt.timedelta(days = 2)) 
if dt.date.today().weekday() == 0: #if monday
    startDate = (dt.date.today() - dt.timedelta(days = 3)) 

endDate = dt.date.today()
# symbol="NSE:TATASTEEL-EQ"
resolution = "5"
technical_df = dynamicDataFetching(fyers,startDate,endDate,symbol,resolution)
clean_data(technical_df)
technical_df = technical_df.rename(columns={"open":"Open","low":"Low","high":"High","close":"Close","volume":"Volume"})

In [90]:
technical_df.shape

(0, 5)

In [66]:
technical_signal = ema_crossover(technical_df)

Now we have both trend_signal and technical_signal, we can have final singal

In [67]:

final_signal = 0
if trend_signal == 2 and technical_signal == 2:
  final_signal = 1 #BUY
elif trend_signal == 1 and technical_signal == 1:
  final_signal = -1 #SELL

In [72]:

positions = fyers.positions()
#  check if already in position
#  check if done for the day or not
can_trade = (already_in_position(positions)==False) and (check_three_loses(positions)==False)

# Taking trades
if can_trade & final_signal != 0:
    stop_loss = 2*technical_df.ATR[-1]
    target = 2*stop_loss
    data = {
        "symbol":symbol,
        "qty":1,
        "type":2,
        "side":final_signal,
        "productType":"BO",
        "limitPrice":0,
        "stopPrice":0,
        "validity":"DAY",
        "disclosedQty":0,
        "offlineOrder":"False",
        "stopLoss":stop_loss,
        "takeProfit":target
    }
    resp = fyers.place_order(data)
    # if resp["code"] == 1101:
    print("Order placed successfully")
    curr_price = fyers.quotes({"symbols":symbol})
    curr_price = curr_price["d"][0]["v"]["lp"]
    absolute_target = curr_price 
    absolute_sl = curr_price
    if final_signal == 1:
        absolute_target = curr_price+target
        absolute_sl = curr_price-target
    elif final_signal == -1:
        absolute_target = curr_price-target
        absolute_sl = curr_price+target
    
    # storing in db 
    cursor.execute(f"""INSERT INTO tradebook VALUES
    (NOW(),'{symbol}',{curr_price},{absolute_target},{absolute_sl},{final_signal})
    """)
    db.commit()
    # else:
    #     print("Order not placed successfully")
else :
    print("No trade - can_trade & final_signal")

No trade - Can_trade = false
